In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import skimage
import os
from sklearn.metrics import mean_squared_error
from skimage import data, img_as_float
from skimage.restoration import denoise_nl_means
from skimage.measure import compare_ssim as ssim
from random import shuffle
import math
from keras.models import model_from_json
from keras.layers import *
from keras.models import Model
from keras import regularizers
from keras import backend as K
from keras.callbacks import TensorBoard
from keras.layers.core import Lambda
import pickle

from utilities import *
%matplotlib inline


Using TensorFlow backend.


In [2]:
def readcifar(path) : 
    with open(path, 'rb') as f:
        train_set= pickle.load(f)
    return train_set["data"]

In [3]:
raw_train_set = readcifar('data/cifar-10-batches-py/data_batch_1')

In [4]:
raw_train_set.shape

(10000, 3072)

In [5]:
def preprocess(images) :
    images = images.astype('float32')/255.
    images = np.reshape(images, (images.shape[0],32,32,3))
    return images

In [6]:
train_set = preprocess(raw_train_set)

In [7]:
train_set.shape

(10000, 32, 32, 3)

In [8]:
def visualise_cifar10():
    from six.moves import cPickle 

    f = open('data/cifar-10-batches-py/data_batch_1', 'rb')
    datadict = cPickle.load(f)
    f.close()
    X = datadict["data"] 
    Y = datadict['labels']
    X = X.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype("uint8")
    Y = np.array(Y)

    #Visualizing CIFAR 10
    fig, axes1 = plt.subplots(5,5,figsize=(3,3))
    for j in range(5):
        for k in range(5):
            i = np.random.choice(range(len(X)))
            axes1[j][k].set_axis_off()
            axes1[j][k].imshow(X[i:i+1][0])
            
#Display last 'n' images from object 'images'
def display_colour_images(images,n,sizeX,sizeY):
    size = images.shape[0]
    plt.figure(figsize=(20, 2))
    for i in range(1,n):
        ax = plt.subplot(1, n, i)
        plt.imshow(images[size - i].reshape(sizeX, sizeY,3))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()

In [9]:
"""visualise_cifar10()"""

'visualise_cifar10()'

In [10]:
display_colour_images(train_set,10,32,32)

RuntimeError: Could not create write struct

In [12]:
#Adding noise to images
def add_noise(images):
    #x_train = np.reshape(x_train, (len(x_train), 64*64))  # adapt this if using `channels_first` image data format
    #x_test = np.reshape(x_test, (len(x_test), 64*64))  # adapt this if using `channels_first` image data format
    batch = images.shape[0]//4;
    noise1 = gaussian_noise(images[0:batch],0,1,0.1)
    noise2 = gaussian_noise(images[batch:2*batch],0,1,0.5)
    noise3 = gaussian_noise(images[2*batch:3*batch],0,2,0.2)
    noise4 = gaussian_noise(images[3*batch:],0,3,0.1)
    
    noisy_set = []
    for data in [noise1,noise2,noise3,noise4]:
        for image in data:
            noisy_set.append(image)
    
    return np.array(noisy_set)
   
#Shuffle the noisy image ground truth pair to randomize the noise distribution in the dataset
def pair_shuffle(images,noisy_set):
    image_pair = []
    for i in range(images.shape[0]):
        image_pair.append((images[i],noisy_set[i]))
    shuffle(image_pair)
    
    ground_truth=[]
    noisy_images = []
    for i in range(images.shape[0]):
        ground_truth.append(image_pair[i][0])
        noisy_images.append(image_pair[i][1])
    return np.array(ground_truth), np.array(noisy_images)

In [13]:
#Shuffling and adding noise to the dataset
shuffle(train_set)
#Getting the noisy image set
noisy_set = add_noise(train_set)
ground_truth,noisy_images = pair_shuffle(train_set,noisy_set)

In [14]:
#Split into training and cross validation
train_size = int(ground_truth.shape[0]*0.8)
x_train = ground_truth[0:train_size]
x_train_noisy = noisy_images[0:train_size]
x_test = ground_truth[train_size:]
x_test_noisy = noisy_images[train_size:]
print (x_train_noisy.shape)
print (x_test_noisy.shape)

(8000, 32, 32, 3)
(2000, 32, 32, 3)


In [15]:
#Ground truth
"""display_images(ground_truth,10)"""

'display_images(ground_truth,10)'

In [16]:
#Noisy images
"""display_images(noisy_images,10)"""

'display_images(noisy_images,10)'

In [38]:
#Defining the model

def get_simple_autoencoder_model(model_path=None):
    
    if(model_path is None):
        autoencoder = None
    else:
        autoencoder = read_model_json(model_path) 
    
    if(autoencoder is None):
        input_img = Input(shape=x_train_noisy[0].shape)  # adapt this if using `channels_first` image data format

        x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
        x = MaxPooling2D((2, 2), padding='same')(x)
        x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
        encoded = MaxPooling2D((2, 2), padding='same')(x)

        x = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
        x = UpSampling2D((2, 2))(x)
        x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
        x = UpSampling2D((2, 2))(x)
        decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

        autoencoder = Model(input_img, decoded)
        autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy',metrics = ['accuracy'])

    print (autoencoder.summary())
    return autoencoder

In [39]:
#Training the model
autoencoder = get_simple_autoencoder_model()
autoencoder.fit(x_train_noisy, x_train,
                epochs=1,
                batch_size=10,
                shuffle=True,
                validation_data=(x_test_noisy, x_test),
                callbacks=[TensorBoard(log_dir='/tmp/autoencoder', histogram_freq=0, write_graph=True)])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 32, 32, 64)        1792      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 16, 16, 64)        36928     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 8, 8, 64)          36928     
_________________________________________________________________
up_sampling2d_13 (UpSampling (None, 16, 16, 64)        0         
__________

KeyboardInterrupt: 

In [20]:
#Function to get saved keras model
def read_model_json(jsonfilePath,h5filePath):
    try:
        json_file = open(jsonfilePath, 'r')
        print json_file
        loaded_model_json = json_file.read()
        json_file.close()
        print "hello"
        loaded_model = model_from_json(loaded_model_json)
         
        # load weights into new model
        loaded_model.load_weights(h5filePath)

        return loaded_model
    except:
        return None

In [45]:
def get_gated_connections(gatePercentageFactor,inputLayer):
    gateFactor = Input(tensor = K.variable([gatePercentageFactor]))
    fractionG = Lambda(lambda x: x[0]*x[1])([inputLayer,gateFactor])
    complement = Lambda(lambda x: x[0] - x[1])([inputLayer,fractionG])
    
    return gateFactor,fractionG,complement

#x is conv layer
#y is de-conv layer
#gf is gating factor
#fg is fractional input from gate
#c is complement ie remaining fraction from the gate
#jt joining tensor of convolution layer and previous de-conv layer 

def get_cnn_dsc_architecture(model_path=None):
    
    if(model_path is None):
        sym_autoencoder = None
    else:
        sym_autoencoder = read_model_json(model_path[0],model_path[1])
        print model_path[0],model_path[1]
    if(sym_autoencoder is None):
        input_img = Input(shape=(32,32,3))  # adapt this if using `channels_first` image data format
        x1 = Conv2D(64, (4, 4), activation='relu', padding='same')(input_img)
        gf1,fg1,c1 = get_gated_connections(0.1,x1)

        x = MaxPooling2D((2, 2), padding='same')(fg1)
        x2 = Conv2D(64, (4, 4), activation='relu', padding='same')(x) 
        gf2,fg2,c2 = get_gated_connections(0.2,x2)

        x = MaxPooling2D((2, 2), padding='same')(fg2)
        x3 = Conv2D(128, (4, 4), activation='relu', padding='same')(x) 
        gf3,fg3,c3 = get_gated_connections(0.3,x3)

        x = MaxPooling2D((2, 2), padding='same')(x3)
        x4 = Conv2D(256, (4, 4), activation='relu', padding='same')(x) 
        gf4,fg4,c4 = get_gated_connections(0.4,x4)

        x = MaxPooling2D((2, 2), padding='same')(x4)
        x5 = Conv2D(512, (4, 4), activation='relu', padding='same')(x) 

        x = UpSampling2D((2, 2))(x5)
        y1 = Conv2DTranspose(256, (4, 4), activation='relu', padding='same')(x) 
        jt4 = Add()([y1,c4])
        x = UpSampling2D((2, 2))(jt4)

        y2 = Conv2DTranspose(128, (4, 4), activation='relu', padding='same')(x) 
        jt3 = Add()([y2,c3])
        x = UpSampling2D((2, 2))(jt3)

        y3 = Conv2DTranspose(64, (4, 4), activation='relu', padding='same')(x) 
        jt2 = Add()([y3,c2])
        x = UpSampling2D((2, 2))(jt2)

        jt1 = Add()([x,c1])
        y4 = Conv2DTranspose(64, (4, 4), activation='relu', padding='same')(jt1)
        y5 = Conv2DTranspose(3, (4, 4), activation='relu', padding='same')(y4) 

        layers = y5

        sym_autoencoder = Model([input_img,gf1,gf2,gf3,gf4],layers)
        sym_autoencoder.compile(optimizer='sgd', loss = 'mean_squared_error', metrics = ['accuracy'])
        print "Model created"
    else:
        print "Saved model loaded"
    print sym_autoencoder.summary()
    return sym_autoencoder

In [98]:
sym_autoencoder = get_cnn_dsc_architecture(("models/CNNDSC.json","models/CNNDSC.h5"))

<open file 'models/CNNDSC.json', mode 'r' at 0x7fc66f17af60>
hello
models/CNNDSC.json models/CNNDSC.h5
Saved model loaded
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 64, 64, 1)     0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 64, 64, 64)    1088        input_1[0][0]                    
____________________________________________________________________________________________________
input_2 (InputLayer)             (1,)                  0                                            
____________________________________________________________________________________________________
lambda_1 (Lambda)                (None, 64, 64, 64)    0           con

In [47]:
sym_autoencoder.fit(x_train_noisy, x_train,
                epochs=1,
                batch_size=10,
                shuffle=True,
                validation_data=(x_test_noisy, x_test),
                callbacks=[TensorBoard(log_dir='/tmp/autoencoder', 
                                       histogram_freq=0,
                                       write_graph=True)])

Train on 8000 samples, validate on 2000 samples
Epoch 1/1
8000/8000 [==============================] - 895s - loss: 0.0491 - acc: 0.3748 - val_loss: 0.0255 - val_acc: 0.3799


In [48]:
x_train_noisy.shape

(8000, 32, 32, 3)

In [49]:
save_model_as_json(sym_autoencoder,'CNNDSC_CIFAR')

In [70]:
#Getting images to test
def get_testing_images(filePath):
    test_images = read_dataset(filePath)
    test_images = np.array(test_images)
    test_images = test_images.astype('float32')/255.
    test_images = np.reshape(test_images, (test_images.shape[0],32,32,3))
    noisy_test_images = add_noise(test_images)
    print (noisy_test_images.shape)
    return (test_images,noisy_test_images)

In [71]:
with open('data/cifar-10-batches-py/test_batch', 'rb') as f:
    test_set= pickle.load(f) 
test_images = test_set["data"]

print(test_images.shape)
test_images = test_images.astype('float32')/255.
test_images = np.reshape(train_set, (test_images.shape[0],32,32,3))
noisy_test_images = add_noise(test_images)

In [64]:
#Denoising using CNN symmetric Autoencoders
out_sym_autoencoder = sym_autoencoder.predict(noisy_test_images,verbose=1)
print (out_sym_autoencoder.shape)

10000/10000 [==============================] - 402s   
(10000, 32, 32, 3)


In [65]:
#Denoising using Autoencoders
out_autoencoder = autoencoder.predict(noisy_test_images,verbose=1)
print (out_autoencoder.shape)

10000/10000 [==============================] - 67s    
(10000, 32, 32, 3)


In [57]:
"""
display_images(test_images,10)
display_images(noisy_test_images,10)
display_images(out,10)
"""

'\ndisplay_images(test_images,10)\ndisplay_images(noisy_test_images,10)\ndisplay_images(out,10)\n'

In [59]:
from sklearn.metrics import mean_squared_error
def get_psnr(imageA,imageB):
    maxI = 1.0
    try:
        return 20*math.log10(maxI) - 10*math.log10(mean_squared_error(imageA.flatten(),imageB.flatten()))
    except:
        return 20*math.log10(maxI)

def get_psnr_result(x_test, out):
    psnr_sum = 0
    for i in range(out.shape[0]):
        psnr_sum += get_psnr(x_test[i].reshape(32,32,3),out[i].reshape(32,32,3))
        
    return 1.0*psnr_sum/out.shape[0];

def get_ssim_result(originalSet,noisySet):
    ssim_sum = 0
    originalSet = originalSet.reshape(originalSet.shape[0],32,32,3)
    noisySet = noisySet.reshape(noisySet.shape[0],32,32,3)
    for i in range(originalSet.shape[0]):
        ssim_sum += ssim(originalSet[i], noisySet[i],data_range=originalSet[i].max() - noisySet[i].min())
    return 1.0*ssim_sum/originalSet.shape[0]

In [85]:
def bm3d_denoise(noisy_image):
    noisy_image = noisy_image.reshape(noisy_image.shape[0],32,32,3)
    noisy_image = noisy_image*255.0
    denoised = []
    for i in range(noisy_image.shape[0]):
        Basic_img = bm3d.BM3D_1st_step(noisy_image[i])
        Final_img = bm3d.BM3D_2nd_step(Basic_img, noisy_image[i])
        denoised.append(Final_img)
        print ("Image " + str(i) + " denoised")
    return np.array(denoised)

In [96]:
def nlm_denoise(noisy_image):
    noisy_image = noisy_image.reshape(noisy_image.shape[0],32,32,3)
    noisy_image = noisy_image*255.0
    denoised = []
    count = 1
    
    for image in noisy_image:
        denoised_image = denoise_nl_means(image, 7, 11, 0.5,multichannel = True)
        denoised.append(denoised_image)
        if(count%100 == 0) :
            print(str(count)+" images denoised")
        count+=1
    return np.array(denoised)

In [66]:
print (test_images.shape,out_autoencoder.shape)
print (mean_squared_error(test_images[0].flatten(),out_autoencoder[0].flatten()))
get_psnr_result(out_autoencoder,test_images)

((10000, 32, 32, 3), (10000, 32, 32, 3))
0.0251461


15.58063056672913

In [67]:
print (test_images.shape,out_sym_autoencoder.shape)
print (mean_squared_error(test_images[0].flatten(),out_sym_autoencoder[0].flatten()))
get_psnr_result(out_sym_autoencoder,test_images)

((10000, 32, 32, 3), (10000, 32, 32, 3))
0.0145239


16.320247602321036

In [68]:
import bm3d

In [86]:
noisy_test_images.shape
bm3d_out = bm3d_denoise(noisy_test_images[0:10])

ValueError: too many values to unpack

In [ ]:
bm3d_out_norm = bm3d_out.astype('float32')/255.0

In [ ]:
get_psnr_result(bm3d_out_norm,test_images)

In [97]:
print(noisy_test_images.shape)
nlm_out = nlm_denoise(noisy_test_images)
nlm_out = nlm_out.astype('float32')/255.0

(10000, 32, 32, 3)
100 images denoised
200 images denoised
300 images denoised
400 images denoised
500 images denoised
600 images denoised
700 images denoised


KeyboardInterrupt: 

In [ ]:
get_psnr_result(nlm_out,test_images)

In [ ]:
get_ssim_result(test_images,out)

In [ ]:
get_ssim_result(test_images,bm3d_out_norm)

In [ ]:
get_ssim_result(test_images,nlm_out)